In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [55]:
web = 'https://www.audible.com/search'
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(options=options)
driver.get(web)

# Espera implícita para que los elementos se carguen
driver.implicitly_wait(10)



In [56]:
#Pagainacion

book_title = []
book_author = []
book_length = []
current_page = 1

# Localiza de la paginacion 
pagination = driver.find_element(By.XPATH, './/ul[contains(@class, "pagingElements")]')
# Listas de paginas
pages = pagination.find_elements(By.TAG_NAME, 'li')
last_page = int(pages[-2].text)

In [57]:
while current_page <= last_page: 
     # Espera implicita
    #time.sleep(2)
    # Espera explicita
    # container = driver.find_element_by_class_name('adbl-impression-container ')
    container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'adbl-impression-container ')))
    # products = container.find_elements_by_xpath('./li')
    products = WebDriverWait(container, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/li[contains(@class, "productListItem")]')))
    # products = container.find_elements_by_xpath('./li')
    # Inicializar el almacenamiento
    # Hacer un "bucle for" a la lista de productos (cada "product" representa un audiolibro)
    for product in products:
        # Usamos la funcion "contains" para buscar elementos que contienen un texto en particular, así evitamos construir XPath largos
        book_title.append(product.find_element(By.XPATH,'.//h3[contains(@class, "bc-heading")]').text)  # Almacenando data en la lista
        book_author.append(product.find_element(By.XPATH,'.//li[contains(@class, "authorLabel")]').text)
        book_length.append(product.find_element(By.XPATH,'.//li[contains(@class, "runtimeLabel")]').text)
        
    current_page += 1

    try:
        next_page = driver.find_element(By.XPATH, './/span[contains(@class, "nextButton")]')
        next_page.click()
    except:
        pass




driver.quit()
# Almacenando data en un dataframe y exportando a un archivo CSV
df_books = pd.DataFrame({'title': book_title, 'author': book_author, 'length': book_length})
df_books.to_csv('Allbooks.csv', index=False)